In [1]:
# 超时
# 仅当做练习线段树
class Node:

    def __init__(self):
        self.val = 0  # 该节点范围内的和
        self.left = None
        self.right = None
        self.lazy = None


class SegmentTree:

    def __init__(self):
        self.root = Node()

    def pushDown(self, node: Node, leftSz, rightSz):
        if not node.left: node.left = Node()
        if not node.right: node.right = Node()
        if node.lazy is None:
            return
        node.left.val += leftSz * node.lazy
        node.right.val += rightSz * node.lazy
        node.left.lazy = node.lazy
        node.right.lazy = node.lazy
        node.lazy = None

    def update(self, node: Node, si, ei, l, r, val):
        if ei < l or si > r:
            return
        if l <= si <= ei <= r:
            node.val += (ei - si + 1) * val
            node.lazy = val
            return
        mi = si + ei >> 1
        self.pushDown(node, mi - si + 1, ei - mi)
        if mi >= l:
            self.update(node.left, si, mi, l, r, val)
        if mi + 1 <= r:
            self.update(node.right, mi + 1, ei, l, r, val)
        node.val = node.left.val + node.right.val

    def query(self, node: Node, si, ei, l, r):
        if ei < l or si > r:
            return 0
        if l <= si <= ei <= r:
            return node.val
        mi = si + ei >> 1
        self.pushDown(node, mi - si + 1, ei - mi)
        ans = 0
        if mi >= l:
            ans += self.query(node.left, si, mi, l, r)
        if mi + 1 <= r:
            ans += self.query(node.right, mi + 1, ei, l, r)
        return ans


class RecentCounter:

    def __init__(self):
        self.st = SegmentTree()

    def ping(self, t: int) -> int:
        self.st.update(self.st.root, 1, 10**9, t, t, 1)
        return self.st.query(self.st.root, 1, 10**9, t - 3000, t)


# Your RecentCounter object will be instantiated and called as such:
# obj = RecentCounter()
# param_1 = obj.ping(t)

In [ ]:
# 超时
# 仅当做练习线段树
# 线段树模板 以增加区间所有值 计算区间和为例的动态开点的线段树
# 线段树的每个节点代表一个区间 节点的值就是该区间的和(根据题目问题改变其含义)
# 数字之和「总数字之和 = 左区间数字之和 + 右区间数字之和」
# 最大公因数 (GCD)「总 GCD = gcd(左区间 GCD, 右区间 GCD)」
# 最大值「总最大值 = max(左区间最大值，右区间最大值)」
class Node:

    def __init__(self, si, ei):
        self.si = si
        self.ei = ei
        self.mi = si + ei >> 1
        self.val = 0
        self.left = None
        self.right = None
        self.lazy = 0
        # lazy标记含义是 该节点已经做出改变 但子节点未下推改变
        # 需要在下推lazy标记方法中进行 动态开点和下推lazy标记 在每次更新和区间和查询区间时候进行下推lazy标记


class SegmentTree:

    def __init__(self, si, ei):  # 数据大范围 如[1,10**9]
        self.root = Node(si, ei)

    def update(self, node: Node, l, r, val):
        '''
        节点node范围[si,ei]
        更新[l,r]所有值增加val
        '''
        if node.ei < l or node.si > r:  # 出界
            return
        if l <= node.si <= node.ei <= r:  # 重叠区域 改变值 标记lazy
            node.val += val
            node.lazy = val
            return
        self.pushDown(node)  # 下推lazy标记和动态开点
        if node.mi >= l:  # 左树与目标区间[l,r]有交集
            self.update(node.left, l, r, val)  # 左树递归找重叠区间增加val
        if node.mi + 1 <= r:  # 右树同理
            self.update(node.right, l, r, val)
        node.val = node.left.val + node.right.val  # 最后改变当前区间值

    def query(self, node: Node, l, r):
        '''
        节点node范围[si,ei]
        查询[l,r]所有值的和
        '''
        if node.ei < l or node.si > r:
            return 0
        if l <= node.si <= node.ei <= r:
            return node.val
        self.pushDown(node)  # 下推lazy标记和动态开点
        ans = 0
        if node.mi >= l:  # 左树与目标区间[l,r]有交集
            ans += self.query(node.left, l, r)  # 左树递归找重叠区间贡献和
        if node.mi + 1 <= r:  # 右树同理
            ans += self.query(node.right, l, r)
        return ans

    def pushDown(self, node: Node):
        # 1.动态开点
        if not node.left:
            node.left = Node(node.si, node.mi)
        if not node.right:
            node.right = Node(node.mi + 1, node.ei)
        # lazy未标记 或 标记为0 无需下推
        if node.lazy == 0:
            return
        node.left.val += (node.left.ei - node.left.si + 1) * node.lazy  # 下推左树
        node.right.val += (node.right.ei - node.right.si + 1) * node.lazy  # 下推lazy
        node.left.lazy = node.lazy  # 下推右树
        node.right.lazy = node.lazy  # 下推lazy
        node.lazy = 0  # 取消当前节点lazy标记


class RecentCounter:

    def __init__(self):
        self.st = SegmentTree(1, 10**9)

    def ping(self, t: int) -> int:
        self.st.update(self.st.root, t, t, 1)
        return self.st.query(self.st.root, t - 3000, t)


# Your RecentCounter object will be instantiated and called as such:
# obj = RecentCounter()
# param_1 = obj.ping(t)